In [298]:
import os
import re
import time
from io import StringIO

from dotenv import load_dotenv

import pandas as pd

from tqdm.auto import tqdm

import requests
from bs4 import BeautifulSoup

from sentence_transformers import SentenceTransformer

import tiktoken

from elasticsearch import Elasticsearch


In [297]:
#!pip install elasticsearch 

   ---------------------------------------- 0.0/524.6 kB ? eta -:--:--
   ---------------------------------------- 524.6/524.6 kB 4.2 MB/s eta 0:00:00


# 

In [4]:
load_dotenv()

True

# 

# 

# Scrapping the wiki page

In [47]:
# Function for scrapping any wikipedia page and save the raw html data
def scrape_wikipedia_page(url,raw_data_filepath='../data/raw/'):

    response = requests.get(url)

    raw_data_filename = re.search(r'wiki/(.*)', url).group().replace('/', '_')
    raw_file_path = f'{raw_data_filepath}{raw_data_filename}.html'

    os.makedirs('data/raw', exist_ok=True)
    with open(raw_file_path, 'w', encoding='utf-8') as file:
        file.write(response.text)


    return response.text

In [38]:
# Function for scrapping any html from wikipedia and extract each content with it related format (Headers, Paragraphs and tables)
def scrape_wikipedia_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extracting the main content of the wiki
    content = soup.find('div', {'class': 'mw-parser-output'})
    
    # Extracting all the formats of the content
    extracted_content = content.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'table'])
    
    return extracted_content

In [124]:
# Function for standard text cleaning. Cleaning special characters and normalizing the text.
def clean_text(text):

    clean_text = re.sub(r'\s+', ' ', text)  # Cleaning extra whitespaces
    #clean_text = re.sub(r'[^\w\s]', '', clean_text)  # Cleaning special characters
    clean_text = clean_text.strip() # Cleaning whitespaces at the start and at the end

    return clean_text


In [113]:
# Function for counting tokens in a text
def count_tokens(text):
    """Función para contar el número de tokens en un texto."""
    return len(encoding.encode(text))

In [109]:
# Aux function for processing the headers when chunking an HTML.
# The headers will be used as an important feature to store with the chunk content for improving the RAG accuracy-
def chunking_processing_HTML_headers(elem, current_headers):
    level = int(elem.name[1])
    current_headers[f'h{level}'] = elem.text.strip()
    
    # Cleaning headers with less level
    for i in range(level + 1, 7):
        current_headers[f'h{i}'] = ''

    return current_headers

In [269]:
# Aux function for processing  the paragraphs when chunking an HTML.
# It needs the info of the actual chunk, actual chunk size, max chunk size and headers.
# It process the paragraph. If the chunk size with the current paragraph will be larger than the max chunk size,
# the current chunk is stored with the relevant info, and a new chunk is created is created from the actual paragraph info. 
# If the chunk size is smaller than the max chunk size, then the paragraph is appended to the current chunk, updating the relevant info.
def chunking_processing_HTML_paragraphs(paragraph, current_headers, current_chunk, current_token_count, max_chunk_size, chunks, ingestion_timestamp, chunk_incremental_counter, source_url):

    headers_concat = " > ".join([current_headers[f'h{i}'] for i in range(1, 7) if current_headers[f'h{i}']])
    chunk_text = paragraph

    tokens_in_chunk = count_tokens(chunk_text)
    
    if current_token_count + tokens_in_chunk <= max_chunk_size:
        current_chunk += chunk_text + " "
        current_token_count += tokens_in_chunk
    else:
        # Add the chunk for storing and reset the information
        chunks.append({
            "content": current_chunk.strip(),             
            "headers_concat": headers_concat,  
            "chunk_size": current_token_count+tokens_in_chunk,     
            "source_url": source_url,                 
            "content_type": "paragraph",
            "ingestion_date": ingestion_timestamp,
            "chunk_id": f"{ingestion_timestamp}_{chunk_incremental_counter:06d}"
            })
        
        chunk_incremental_counter+=1
        current_chunk = chunk_text + " "
        current_token_count = tokens_in_chunk

    return current_chunk, current_token_count, chunks, headers_concat, chunk_incremental_counter

In [273]:
# Aux function for processing  the tables when chunking an HTML.
# It needs the info of the actual chunk, actual chunk size, max chunk size and headers.
# It process the table in a single chunk for avoiding a splitting of the table that could generate misinformation and errors.
def chunking_processing_HTML_tables(table_html, current_headers, current_chunk, current_token_count, max_chunk_size, chunks, ingestion_timestamp, chunk_incremental_counter, source_url):
    try:
        table_df = pd.read_html(StringIO(table_html))[0]
        table_text = table_df.to_string(index=False)
        headers_concat = " > ".join([current_headers[f'h{i}'] for i in range(1, 7) if current_headers[f'h{i}']])
        chunk_text = table_text

        tokens_in_chunk = count_tokens(chunk_text)

        # Handling the chunk with the objective of mantaining the table in a single chunk.
        if current_chunk and current_token_count + tokens_in_chunk > max_chunk_size:
            # If the chunk will be larger, then we will store it first, before storing the table.
            # Add the chunk for storing and reset the information
            chunks.append({
                "content": current_chunk.strip(),             
                "headers_concat": headers_concat,  
                "chunk_size": current_token_count+tokens_in_chunk,     
                "source_url": source_url,                 
                "content_type": "paragraph",
                "ingestion_date": ingestion_timestamp,
                "chunk_id": f"{ingestion_timestamp}_{chunk_incremental_counter:06d}"
                })
            chunk_incremental_counter+=1
            current_chunk = ""  
            current_token_count = 0

        # Add the whole table content in the same chunk
        # Add the chunk for storing and reset the information
        chunks.append({
            "content": chunk_text.strip(),             
            "headers_concat": headers_concat,  
            "chunk_size": current_token_count+tokens_in_chunk,     
            "source_url": source_url,                 
            "content_type": "paragraph+table",
            "ingestion_date": ingestion_timestamp,
            "chunk_id": f"{ingestion_timestamp}_{chunk_incremental_counter:06d}"
            })
        
        chunk_incremental_counter+=1
        current_chunk = ""  # Reset the chunk after adding the table
        current_token_count = 0

    except ValueError:
        headers_concat = " > ".join([current_headers[f'h{i}'] for i in range(1, 7) if current_headers[f'h{i}']])
        chunks.append({'content': 'Error processing the table', 'headers_concat': headers_concat})

    return current_chunk, current_token_count, chunks, headers_concat, chunk_incremental_counter
    

In [271]:
# Function for processing an html and generating chunks in an strategic way based on the best practices.
# It could recieve the max_chunk_size parameter. If a chunk is going to be larger than it, then the chunk will be splitted.

def create_chunks_with_headers(elements, source_url, max_chunk_size=500):

    # Generating the variables used to store with the chunk content.
    ingestion_timestamp = time.strftime("%Y%m%d%H%M%S")
    chunk_incremental_counter = 0


    ################################################
    chunks = []
    current_headers = {f'h{i}': '' for i in range(1, 7)}
    current_chunk = ""
    current_token_count = 0

    # Iterating over the HTML elements and processing the headers, paragraphs, and tables in different ways.
    for elem in elements:

        # If it's a header, we should update the current_header info.
        if elem.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
            current_headers = chunking_processing_HTML_headers(elem, current_headers)

        # If it's a paragraph, whe should update the chubk content info, considering the chunk_size and actual chunk info and size.
        elif elem.name == 'p':
            paragraph = clean_text(elem.text)
            current_chunk, current_token_count, chunks, headers_concat, chunk_incremental_counter = chunking_processing_HTML_paragraphs(paragraph, current_headers, 
                                                                                                                                        current_chunk, current_token_count, 
                                                                                                                                        max_chunk_size, chunks,
                                                                                                                                        ingestion_timestamp, chunk_incremental_counter,
                                                                                                                                        source_url)

        elif elem.name == 'table':
            table_html = str(elem)
            current_chunk, current_token_count, chunks, headers_concat, chunk_incremental_counter = chunking_processing_HTML_tables(table_html, current_headers, 
                                                                                                                                    current_chunk, current_token_count, 
                                                                                                                                    max_chunk_size, chunks,
                                                                                                                                    ingestion_timestamp, chunk_incremental_counter,
                                                                                                                                    source_url)

    # Adding the last chunk
    if current_chunk:
        chunks.append({'content': current_chunk.strip(), 'headers_concat': headers_concat})

    return chunks

## 

## 

## 

## 

We will start defining general variables used in the functions and the ingestion step of the pipeline

In [115]:
# standard encoding for the tiktoken tokens count
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# url to be processed for the ingestion
url = "https://es.wikipedia.org/wiki/Lionel_Messi"

Processing the content from the Lionel Messi Wikipedia. 

In a future it could be used with any wikipedia page, adapting the APP allowing an user to add a wiki URL for creating an assistant expert on it page.

In [51]:
html_content = scrape_wikipedia_page(url)
processed_html_content = scrape_wikipedia_html(html_content)


Now, we will start with the chunking step for store them into a vector database.

Thinking about best practices of chunking strategies we know:
- **Semantic context**: We need to keep the semantic coherence of the content. A too short chunk could be resulting in a context loss model. We need to use the headers and tables in a strategic way.
- **Chunk Size**: The chunk size should be efficient for processing and context. We know that the suggested chunks size is between 200-500 tokens. Qe can use libraries like tiktoken to monitor it.
- **Heading grouping**: The headers from the html will be very helpful for defining natural sections of the contents.
- **Tables**: The tables are natural structured info. It's a good practice to use them as a separate chunk.

We'll adopt a kind of an advanced chunking strategy.

We will adopt the best practices and we chunk all the html content considering chunks with less tan 500 tokens, ensuring that all a whole table is in the same chunk, and generating semantic and heading grouping stored with the chunk content like headers, content type, IDs, url, chunk_size, among others

In [275]:
chunks = create_chunks_with_headers(processed_html_content, source_url=url, max_chunk_size=500)

In [278]:
chunks[3]

{'content': 'En 2009, a los veintidós años, ganó su primer Balón de Oro y el premio al Jugador Mundial de la FIFA del año. Siguieron tres temporadas exitosas, en las que ganó cuatro Balones de Oro de forma consecutiva, hecho que no tenía precedentes. Hasta el momento, su mejor campaña personal fue en 2011-12, cuando estableció el récord de más goles en una temporada, tanto en La Liga como en otras competiciones europeas. Durante las dos siguientes temporadas, también sufrió lesiones y, en 2014, perdió el Balón de Oro frente a Cristiano Ronaldo, a quien se considera su rival. Recuperó su mejor forma durante la campaña 2014-15, en la que superó los registros de máximo goleador absoluto en La Liga y la Liga de Campeones y logró con el Barcelona un histórico segundo triplete, además de ganar su quinto Balón de Oro. Volvería a ganarlo en 2019, 2021 y 2023. Como internacional argentino, ha representado a su país en catorce torneos mayores. A nivel juvenil, en 2005 participó con la selección 

In [276]:
print(f"TOTAL CHUNKS = {len(chunks)}")
print(f"Max chunk size: {max(chunks, key=lambda x: x['chunk_size'])['chunk_size']}")
print(f"Min chunk size: {min(chunks, key=lambda x: x['chunk_size'])['chunk_size']}")
print(f"avg chunk size: {sum(chunk['chunk_size'] for chunk in chunks) / len(chunks)}")

TOTAL CHUNKS = 110
Max chunk size: 4152
Min chunk size: 44
avg chunk size: 622.5727272727273


Now, we'll add the embeddings to each chunk for simmilarity search. This way we can try text search and vector search.

In [308]:
# Initialize the embeddings model
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\TALIGENT\Desktop\Proyectos\LLM-Zoomcamp\MessiXpert-RAG\messiXpert-RAG-venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [363]:
# Adding the embedding for each chunk content
for chunk in tqdm(chunks):

    # Generating the embeddings
    embedding = embeddings_model.encode(chunk['content']).tolist()
    
    # Adding the embeddings to the original dict
    chunk['content_embeddings'] = embedding

    

  0%|          | 0/110 [00:00<?, ?it/s]

Now we'll create the vectorDB and store the chunks in the DB.

For this project, we'll use Weaviate. 

Weaviate has Native support for different search metrics like euclidean, cosine simmillarity, etc.
Weaviate also has different search methods like KNN, ANN, and it supports an integration with text search.

docker run -p 127.0.0.1:9200:9200 -d --name elasticsearch -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.license.self_generated.type=trial" -v "elasticsearch-data:/usr/share/elasticsearch/data" docker.elastic.co/elasticsearch/elasticsearch:8.15.0


In [329]:
# Initialize the Elasticsearch client
es_client = Elasticsearch("http://localhost:9200")

In [331]:
# Creating the index settings for the vectorDB. We will define two different indexes for testing purpose. Cosine and dot product.
# The final version, and the .py associated, will have the best one.
index_settings_cosine = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "content": {"type": "text"},
            "headers_concat": {"type": "text"},
            "chunk_size": {"type": "integer"},
            "source_url": {"type": "text"} ,
            "content_type": {"type": "text"} ,
            "ingestion_date": {"type": "date"} ,
            "content_embeddings": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}


index_settings_dot_product = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "content": {"type": "text"},
            "headers_concat": {"type": "text"},
            "chunk_size": {"type": "integer"},
            "source_url": {"type": "text"} ,
            "content_type": {"type": "text"} ,
            "ingestion_date": {"type": "date"} ,
            "content_embeddings": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "dot_product"},
        }
    }
}

In [333]:
# Creating the index in ElasticSearch
index_name_cosine = "messixpert_cosine"
index_name_dot_product = "messixpert_dot_product"


es_client.indices.create(index=index_name_cosine, body=index_settings_cosine)
es_client.indices.create(index=index_name_dot_product, body=index_settings_dot_product)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'messixpert_dot_product'})

In [335]:
for chunk in tqdm(chunks):
    try:
        es_client.index(index=index_name_cosine, document=chunk)
    except Exception as e:
        print(e)

In [336]:
for chunk in tqdm(chunks):
    try:
        es_client.index(index=index_name_dot_product, document=chunk)
    except Exception as e:
        print(e)

We'll simply test each index with two different questions for each

In [346]:
search_term_1 = "Cuantos hermanos tiene Messi?"
vector_search_term_1 = embeddings_model.encode(search_term_1)

search_term_2 = "En que clubes jugó profesionalmente Messi?"
vector_search_term_2 = embeddings_model.encode(search_term_2)

In [350]:
query_1 = {
    "k": 5,
    "field": "content_embeddings",
    "query_vector": vector_search_term_1,
    "num_candidates": 10000, 
}

query_2 = {
    "k": 5,
    "field": "content_embeddings",
    "query_vector": vector_search_term_2,
    "num_candidates": 10000, 
}

In [351]:
res_cosine_1 = es_client.search(index=index_name_cosine, knn=query_1, source=["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"])
res_cosine_1["hits"]["hits"]

[{'_index': 'messixpert_cosine',
  '_id': 'jauLFpIBRGyVeHbOft23',
  '_score': 0.7673075,
  '_source': {'content': '0\nMe hubiera gustado jugar con Messi—Pelé en una entrevista con La Gazzetta dello Sport en noviembre de 2019 para conmemorar los cincuenta años de su gol número mil.[344]\u200b',
   'headers_concat': 'Trayectoria > Titularidad y consagración en la élite > Dominio local con Ernesto Valverde y años difíciles en Champions > 2019-2020: sexto Balón de Oro y Premio Laureus',
   'chunk_size': 199,
   'source_url': 'https://es.wikipedia.org/wiki/Lionel_Messi',
   'content_type': 'paragraph+table',
   'ingestion_date': '20240921163112'}},
 {'_index': 'messixpert_cosine',
  '_id': 'lauLFpIBRGyVeHbOf92J',
  '_score': 0.76242065,
  '_source': {'content': '0\nMessi será el jugador que más Balones de Oro gane en la historia. Probablemente ganará cinco, seis o siete Balones de Oro, es imparable.—Johan Cruyff a Olé en 2011[427]\u200b',
   'headers_concat': 'Trayectoria > Paris Saint-Germ

In [352]:
res_dotproduct_1 = es_client.search(index=index_name_dot_product, knn=query_1, source=["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"])
res_dotproduct_1["hits"]["hits"]

[{'_index': 'messixpert_dot_product',
  '_id': '-6uLFpIBRGyVeHbOtN1B',
  '_score': 0.76728725,
  '_source': {'content': '0\nMe hubiera gustado jugar con Messi—Pelé en una entrevista con La Gazzetta dello Sport en noviembre de 2019 para conmemorar los cincuenta años de su gol número mil.[344]\u200b',
   'headers_concat': 'Trayectoria > Titularidad y consagración en la élite > Dominio local con Ernesto Valverde y años difíciles en Champions > 2019-2020: sexto Balón de Oro y Premio Laureus',
   'chunk_size': 199,
   'source_url': 'https://es.wikipedia.org/wiki/Lionel_Messi',
   'content_type': 'paragraph+table',
   'ingestion_date': '20240921163112'}},
 {'_index': 'messixpert_dot_product',
  '_id': 'A6uLFpIBRGyVeHbOtN79',
  '_score': 0.7629819,
  '_source': {'content': '0\nMessi será el jugador que más Balones de Oro gane en la historia. Probablemente ganará cinco, seis o siete Balones de Oro, es imparable.—Johan Cruyff a Olé en 2011[427]\u200b',
   'headers_concat': 'Trayectoria > Paris 

In [353]:
res_cosine_2 = es_client.search(index=index_name_cosine, knn=query_2, source=["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"])
res_cosine_2["hits"]["hits"]

[{'_index': 'messixpert_cosine',
  '_id': 'dKuLFpIBRGyVeHbOfN0U',
  '_score': 0.8388665,
  '_source': {'content': "Messi pudo jugar en La Liga el 1 de octubre, en el empate 2-2 con el Zaragoza, donde entró en el minuto '66 en lugar de Giuly.[73]\u200b El Alavés y el Deportivo La Coruña denunciaron ante el Comité de Competición su inclusión en la plantilla, por considerar que se le había dado la licencia como juvenil y no como profesional. El Comité, sin embargo, declaró improcedente la denuncia, ya que el jugador conservaba la licencia de juvenil y era ciudadano español.[74]\u200b El 2 de noviembre, Messi convirtió su primer gol en Liga de Campeones ante el Panathinaikos, al que el Barcelona le ganó 5-0.[75]\u200b El 19, por un encuentro de La Liga en el estadio Santiago Bernabéu, fue titular en su primer Clásico, que perdió el Real Madrid 0-3 y en el que Ronaldinho recibió aplausos de la afición rival. Dio el pase a Eto'o para el primer gol, superó el marcaje de Roberto Carlos y acele

In [354]:
res_dotproduct_2 = es_client.search(index=index_name_dot_product, knn=query_2, source=["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"])
res_dotproduct_2["hits"]["hits"]

[{'_index': 'messixpert_dot_product',
  '_id': '4quLFpIBRGyVeHbOsd31',
  '_score': 0.83841205,
  '_source': {'content': "Messi pudo jugar en La Liga el 1 de octubre, en el empate 2-2 con el Zaragoza, donde entró en el minuto '66 en lugar de Giuly.[73]\u200b El Alavés y el Deportivo La Coruña denunciaron ante el Comité de Competición su inclusión en la plantilla, por considerar que se le había dado la licencia como juvenil y no como profesional. El Comité, sin embargo, declaró improcedente la denuncia, ya que el jugador conservaba la licencia de juvenil y era ciudadano español.[74]\u200b El 2 de noviembre, Messi convirtió su primer gol en Liga de Campeones ante el Panathinaikos, al que el Barcelona le ganó 5-0.[75]\u200b El 19, por un encuentro de La Liga en el estadio Santiago Bernabéu, fue titular en su primer Clásico, que perdió el Real Madrid 0-3 y en el que Ronaldinho recibió aplausos de la afición rival. Dio el pase a Eto'o para el primer gol, superó el marcaje de Roberto Carlos y

Now we'll also try a text search

In [ ]:
query_2 = {
    "k": 5,
    "field": "content_embeddings",
    "query_vector": vector_search_term_2,
    "num_candidates": 10000, 
}
res_cosine_1 = es_client.search(index=index_name_cosine, knn=query_1, source=["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"])
res_cosine_1["hits"]["hits"]

In [358]:
text_query_1 = {
    "size": 5,  
    "_source": ["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"],  
    "query": {
        "match": {
            "content": search_term_1  
        }
    }
}

text_query_2 = {
    "size": 5,  
    "_source": ["content", "headers_concat", "chunk_size", "source_url", "content_type", "ingestion_date"],  
    "query": {
        "match": {
            "content": search_term_1  
        }
    }
}
res = es_client.search(index=index_name_cosine, body=text_query_1)


In [357]:
res_cosine_text_1 = es_client.search(index=index_name_cosine, body=text_query_1)
res_cosine_text_1["hits"]["hits"]

[{'_index': 'messixpert_cosine',
  '_id': 'aKuLFpIBRGyVeHbOe90G',
  '_score': 7.540992,
  '_source': {'content': 'Lionel Andrés Messi nació el 24 de junio de 1987 en el Hospital Italiano Garibaldi de la ciudad de Rosario, en la provincia de Santa Fe. Es el tercer hijo de Jorge Horacio Messi y Celia María Cuccittini. Tiene dos hermanos mayores, Rodrigo y Matías, y una hermana menor, María Sol.[14]\u200b Su familia paterna es originaria del municipio italiano de Recanati, de donde su bisabuelo, Angelo Messi, emigró a Argentina en 1883.[15]\u200b Fue su abuela materna, Celia, la que lo alentó a dedicarse al fútbol, por lo que él le agradece señalando al cielo con las dos manos tras convertir un gol.[14]\u200b[16]\u200b Dos de sus primos (Maximiliano y Emanuel Biancucchi) son también futbolistas.[14]\u200b Estudió en la escuela primaria N° 66 "Gral. Las Heras".[14]\u200b Con apenas cuatro años, comenzó a practicar fútbol con Salvador Aparicio en el club Abanderado Grandoli, ubicado en el b

In [359]:
res_cosine_text_2 = es_client.search(index=index_name_cosine, body=text_query_2)
res_cosine_text_2["hits"]["hits"]

[{'_index': 'messixpert_cosine',
  '_id': 'aKuLFpIBRGyVeHbOe90G',
  '_score': 7.540992,
  '_source': {'content': 'Lionel Andrés Messi nació el 24 de junio de 1987 en el Hospital Italiano Garibaldi de la ciudad de Rosario, en la provincia de Santa Fe. Es el tercer hijo de Jorge Horacio Messi y Celia María Cuccittini. Tiene dos hermanos mayores, Rodrigo y Matías, y una hermana menor, María Sol.[14]\u200b Su familia paterna es originaria del municipio italiano de Recanati, de donde su bisabuelo, Angelo Messi, emigró a Argentina en 1883.[15]\u200b Fue su abuela materna, Celia, la que lo alentó a dedicarse al fútbol, por lo que él le agradece señalando al cielo con las dos manos tras convertir un gol.[14]\u200b[16]\u200b Dos de sus primos (Maximiliano y Emanuel Biancucchi) son también futbolistas.[14]\u200b Estudió en la escuela primaria N° 66 "Gral. Las Heras".[14]\u200b Con apenas cuatro años, comenzó a practicar fútbol con Salvador Aparicio en el club Abanderado Grandoli, ubicado en el b

In [361]:
res_dot_product_text_1 = es_client.search(index=index_name_dot_product, body=text_query_1)
res_dot_product_text_1["hits"]["hits"]

[{'_index': 'messixpert_dot_product',
  '_id': '1quLFpIBRGyVeHbOsN3o',
  '_score': 7.540992,
  '_source': {'content': 'Lionel Andrés Messi nació el 24 de junio de 1987 en el Hospital Italiano Garibaldi de la ciudad de Rosario, en la provincia de Santa Fe. Es el tercer hijo de Jorge Horacio Messi y Celia María Cuccittini. Tiene dos hermanos mayores, Rodrigo y Matías, y una hermana menor, María Sol.[14]\u200b Su familia paterna es originaria del municipio italiano de Recanati, de donde su bisabuelo, Angelo Messi, emigró a Argentina en 1883.[15]\u200b Fue su abuela materna, Celia, la que lo alentó a dedicarse al fútbol, por lo que él le agradece señalando al cielo con las dos manos tras convertir un gol.[14]\u200b[16]\u200b Dos de sus primos (Maximiliano y Emanuel Biancucchi) son también futbolistas.[14]\u200b Estudió en la escuela primaria N° 66 "Gral. Las Heras".[14]\u200b Con apenas cuatro años, comenzó a practicar fútbol con Salvador Aparicio en el club Abanderado Grandoli, ubicado en

In [362]:
res_dot_product_text_2 = es_client.search(index=index_name_dot_product, body=text_query_2)
res_dot_product_text_2["hits"]["hits"]

[{'_index': 'messixpert_dot_product',
  '_id': '1quLFpIBRGyVeHbOsN3o',
  '_score': 7.540992,
  '_source': {'content': 'Lionel Andrés Messi nació el 24 de junio de 1987 en el Hospital Italiano Garibaldi de la ciudad de Rosario, en la provincia de Santa Fe. Es el tercer hijo de Jorge Horacio Messi y Celia María Cuccittini. Tiene dos hermanos mayores, Rodrigo y Matías, y una hermana menor, María Sol.[14]\u200b Su familia paterna es originaria del municipio italiano de Recanati, de donde su bisabuelo, Angelo Messi, emigró a Argentina en 1883.[15]\u200b Fue su abuela materna, Celia, la que lo alentó a dedicarse al fútbol, por lo que él le agradece señalando al cielo con las dos manos tras convertir un gol.[14]\u200b[16]\u200b Dos de sus primos (Maximiliano y Emanuel Biancucchi) son también futbolistas.[14]\u200b Estudió en la escuela primaria N° 66 "Gral. Las Heras".[14]\u200b Con apenas cuatro años, comenzó a practicar fútbol con Salvador Aparicio en el club Abanderado Grandoli, ubicado en